### Load
#### Load gluonnlp

In [1]:
import warnings
warnings.filterwarnings('ignore')

import glob
import time
import math

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon.utils import download

import gluonnlp as nlp

#### Set environment

In [2]:
num_gpus = 1
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus else [mx.cpu()]
log_interval = 200

#### Set hyperparameters

In [3]:
batch_size = 20 * len(context)
lr = 20
epochs = 3
bptt = 35
grad_clip = 0.25

#### Load dataset, extract vocabulary, numericalize, and batchify for truncated Back Propagation Through Time (BPTT)

In [4]:
dataset_name = 'wikitext-2'
train_dataset, val_dataset, test_dataset = [
    nlp.data.WikiText2(
        segment=segment, bos=None, eos='<eos>', skip_empty=False)
    for segment in ['train', 'val', 'test']
]

vocab = nlp.Vocab(
    nlp.data.Counter(train_dataset), padding_token=None, bos_token=None)

bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
    vocab, bptt, batch_size, last_batch='discard')
train_data, val_data, test_data = [
    bptt_batchify(x) for x in [train_dataset, val_dataset, test_dataset]
]

#### Load pre-defined language model architecture

In [5]:
model, vocab = nlp.model.awd_lstm_lm_1150(dataset_name='wikitext-2',pretrained=True,ctx=context)
print(model)
print(vocab)

trainer = gluon.Trainer(model.collect_params(), 'sgd', {
    'learning_rate': lr,
    'momentum': 0,
    'wd': 0
})
loss = gluon.loss.SoftmaxCrossEntropyLoss()

AWDRNN(
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 400, float32)
    (1): Dropout(p = 0.65, axes=(0,))
  )
  (encoder): Sequential(
    (0): LSTM(400 -> 1150, TNC)
    (1): LSTM(1150 -> 1150, TNC)
    (2): LSTM(1150 -> 400, TNC)
  )
  (decoder): HybridSequential(
    (0): Dense(400 -> 33278, linear)
  )
)
Vocab(size=33278, unk="<unk>", reserved="['<eos>']")


### Training

Now that everything is ready, we can start training the model.

#### Detach gradients on states for truncated BPTT

In [6]:
def detach(hidden):
    if isinstance(hidden, (tuple, list)):
        hidden = [detach(i) for i in hidden]
    else:
        hidden = hidden.detach()
    return hidden

#### Evaluation

In [7]:
def evaluate(model, data_source, batch_size, ctx):
    total_L = 0.0
    ntotal = 0
    hidden = model.begin_state(batch_size=batch_size, func=mx.nd.zeros, ctx=ctx)
    for i, (data, target) in enumerate(data_source):
        print(data)
        data = data.as_in_context(ctx)
        target = target.as_in_context(ctx)
        output, hidden = model(data, hidden)
        hidden = detach(hidden)
        L = loss(output.reshape(-3, -1), target.reshape(-1))
        total_L += mx.nd.sum(L).asscalar()
        ntotal += L.size
    return total_L / ntotal

#### Training loop

In [ ]:
def train(model, train_data, val_data, test_data, epochs, lr):
    best_val = float("Inf")
    start_train_time = time.time()
    parameters = model.collect_params().values()
    for epoch in range(epochs):
        total_L = 0.0
        start_epoch_time = time.time()
        start_log_interval_time = time.time()
        hiddens = [model.begin_state(batch_size//len(context), func=mx.nd.zeros, ctx=ctx)
                   for ctx in context]
        for i, (data, target) in enumerate(train_data):
            data_list = gluon.utils.split_and_load(data, context,
                                                   batch_axis=1, even_split=True)
            target_list = gluon.utils.split_and_load(target, context,
                                                     batch_axis=1, even_split=True)
            hiddens = detach(hiddens)
            L = 0
            Ls = []
            with autograd.record():
                for j, (X, y, h) in enumerate(zip(data_list, target_list, hiddens)):
                    output, h = model(X, h)
                    batch_L = loss(output.reshape(-3, -1), y.reshape(-1,))
                    L = L + batch_L.as_in_context(context[0]) / (len(context) * X.size)
                    Ls.append(batch_L / (len(context) * X.size))
                    hiddens[j] = h
            L.backward()
            grads = [p.grad(x.context) for p in parameters for x in data_list]
            gluon.utils.clip_global_norm(grads, grad_clip)

            trainer.step(1)

            total_L += sum([mx.nd.sum(l).asscalar() for l in Ls])

            if i % log_interval == 0 and i > 0:
                cur_L = total_L / log_interval
                print('[Epoch %d Batch %d/%d] loss %.2f, ppl %.2f, '
                      'throughput %.2f samples/s'%(
                    epoch, i, len(train_data), cur_L, math.exp(cur_L),
                    batch_size * log_interval / (time.time() - start_log_interval_time)))
                total_L = 0.0
                start_log_interval_time = time.time()

        mx.nd.waitall()

        print('[Epoch %d] throughput %.2f samples/s'%(
                    epoch, len(train_data)*batch_size / (time.time() - start_epoch_time)))
        val_L = evaluate(model, val_data, batch_size, context[0])
        print('[Epoch %d] time cost %.2fs, valid loss %.2f, valid ppl %.2f'%(
            epoch, time.time()-start_epoch_time, val_L, math.exp(val_L)))

        if val_L < best_val:
            best_val = val_L
            test_L = evaluate(model, test_data, batch_size, context[0])
            model.save_parameters('{}_{}-{}.params'.format(model_name, dataset_name, epoch))
            print('test loss %.2f, test ppl %.2f'%(test_L, math.exp(test_L)))
        else:
            lr = lr*0.25
            print('Learning rate now %f'%(lr))
            trainer.set_learning_rate(lr)

    print('Total training throughput %.2f samples/s'%(
                            (batch_size * len(train_data) * epochs) /
                            (time.time() - start_train_time)))

#### Train and evaluate

In [ ]:
train(model, train_data, val_data, test_data, epochs, lr)

## Use pre-trained AWD LSTM language model

AWD LSTM language model is the state-of-the-art RNN language model [1]. The main technique is to add weight-dropout on the recurrent hidden to hidden matrices to prevent overfitting on the recurrent connections.

### Load vocabulary and pre-trained model

In [8]:
awd_model_name = 'awd_lstm_lm_1150'
awd_model, vocab = nlp.model.get_model(
    awd_model_name,
    vocab=vocab,
    dataset_name=dataset_name,
    pretrained=True,
    ctx=context[0])
print(awd_model)
print(vocab)

AWDRNN(
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 400, float32)
    (1): Dropout(p = 0.65, axes=(0,))
  )
  (encoder): Sequential(
    (0): LSTM(400 -> 1150, TNC)
    (1): LSTM(1150 -> 1150, TNC)
    (2): LSTM(1150 -> 400, TNC)
  )
  (decoder): HybridSequential(
    (0): Dense(400 -> 33278, linear)
  )
)
Vocab(size=33278, unk="<unk>", reserved="['<eos>']")


### Evaluate the pre-trained model on val and test datasets

In [9]:
val_L = evaluate(awd_model, val_data, batch_size, context[0])
test_L = evaluate(awd_model, test_data, batch_size, context[0])
print('Best validation loss %.2f, val ppl %.2f' % (val_L, math.exp(val_L)))
print('Best test loss %.2f, test ppl %.2f' % (test_L, math.exp(test_L)))


[[1.0000e+00 3.4000e+01 0.0000e+00 7.0000e+01 3.6000e+01 0.0000e+00
  2.9700e+02 3.0000e+00 3.7640e+03 1.0000e+00 2.1000e+01 2.0000e+00
  2.0100e+02 4.8000e+02 1.1000e+01 5.1900e+02 2.6300e+02 2.0000e+00
  4.0000e+00 3.0410e+03]
 [1.0000e+01 2.0000e+00 3.0770e+03 4.3000e+01 1.1000e+01 4.1000e+01
  3.0000e+00 6.6000e+02 2.1000e+01 1.0000e+00 0.0000e+00 5.5600e+02
  2.8100e+02 4.0000e+00 2.7770e+03 2.3800e+02 8.0000e+00 2.5600e+02
  1.0000e+00 1.6060e+03]
 [1.0452e+04 1.4290e+03 2.4000e+01 2.9300e+02 5.9700e+02 0.0000e+00
  1.9843e+04 1.0300e+02 2.0000e+00 1.0000e+01 2.7000e+01 2.9160e+03
  3.0000e+00 1.0000e+00 1.0140e+03 1.2400e+02 1.0280e+03 9.0000e+00
  1.0000e+00 4.0000e+00]
 [3.1469e+04 6.3990e+03 0.0000e+00 1.2190e+03 7.0000e+00 0.0000e+00
  4.5000e+01 2.6498e+04 4.6400e+02 8.3320e+03 4.8700e+02 8.0000e+00
  2.9000e+01 1.3900e+02 1.1000e+01 6.7200e+02 9.0000e+00 1.8600e+02
  1.0000e+01 5.9060e+03]
 [1.0000e+01 2.0000e+00 3.0000e+00 3.0000e+00 2.6112e+04 0.0000e+00
  9.3600e+02 2.


[[1.7900e+02 3.0000e+00 1.0600e+02 3.4100e+02 3.0000e+00 0.0000e+00
  1.2643e+04 0.0000e+00 2.0000e+00 1.8000e+01 1.5000e+01 5.0000e+00
  2.0240e+03 0.0000e+00 2.5700e+02 1.9000e+01 4.9000e+01 7.0400e+02
  1.1000e+01 1.1210e+03]
 [6.0000e+00 6.9000e+01 9.2890e+03 0.0000e+00 3.3000e+01 4.7680e+03
  6.0000e+00 0.0000e+00 1.3660e+03 0.0000e+00 4.6800e+02 2.0000e+00
  1.4350e+03 2.0889e+04 2.0000e+00 2.8600e+02 3.9070e+03 1.8000e+01
  6.0000e+00 5.0000e+00]
 [1.5599e+04 2.0000e+00 7.0000e+00 0.0000e+00 1.3300e+02 7.0000e+00
  8.5050e+03 1.8560e+03 5.0000e+00 1.7000e+01 2.0000e+01 5.8000e+02
  1.1420e+03 6.0000e+00 1.9229e+04 4.0000e+00 2.1000e+01 3.8000e+01
  1.6000e+01 0.0000e+00]
 [1.6200e+02 1.5100e+02 4.8350e+03 1.1000e+01 7.0000e+00 2.7315e+04
  8.0000e+00 1.5000e+01 2.0227e+04 6.4700e+02 3.8000e+01 3.2000e+01
  2.0000e+00 2.6330e+03 2.8530e+03 0.0000e+00 2.0000e+00 1.7700e+03
  1.0300e+02 3.7000e+01]
 [5.3000e+01 1.7000e+01 4.0000e+00 3.0000e+00 5.0000e+02 8.0200e+02
  3.8000e+01 2.


[[7.0000e+00 5.3000e+01 9.1000e+01 6.3920e+03 2.0000e+00 3.0000e+00
  1.8000e+01 0.0000e+00 1.7080e+03 3.0000e+00 2.8000e+02 3.0000e+00
  4.2400e+02 8.9550e+03 2.0000e+00 5.0000e+00 0.0000e+00 9.0000e+00
  1.0230e+03 1.0000e+00]
 [2.0000e+00 1.1200e+02 9.7100e+02 3.0000e+00 3.3800e+02 1.5500e+02
  2.0000e+00 2.9700e+02 1.1000e+01 3.4700e+02 4.1000e+01 1.6100e+02
  4.5000e+01 6.6400e+02 0.0000e+00 2.0000e+00 2.1000e+01 8.5190e+03
  3.0000e+00 1.0000e+01]
 [2.6340e+03 7.0000e+00 1.6000e+01 9.0000e+00 5.7900e+02 2.2780e+03
  5.5900e+03 4.0000e+00 4.0000e+00 3.0000e+00 2.0000e+00 1.2818e+04
  1.1000e+01 1.4690e+03 1.0140e+03 1.4700e+03 2.0000e+00 4.2500e+02
  0.0000e+00 1.0000e+01]
 [1.9100e+02 1.4290e+03 9.7320e+03 0.0000e+00 3.7000e+01 8.0000e+00
  5.0000e+00 7.4600e+02 1.3557e+04 6.9000e+01 2.0380e+03 3.0000e+00
  1.1850e+04 3.0000e+00 3.0000e+00 5.4500e+02 1.2800e+02 4.6000e+01
  2.1000e+01 1.0000e+01]
 [3.8000e+01 8.0000e+00 6.0000e+00 1.5280e+03 0.0000e+00 5.0230e+03
  1.0080e+03 0.


[[1.9000e+01 1.2000e+01 2.5000e+01 6.8880e+03 1.4000e+01 3.0000e+00
  2.4690e+03 3.5500e+02 6.0000e+00 1.7815e+04 1.9050e+03 2.3351e+04
  2.5590e+03 4.0790e+03 4.0000e+00 0.0000e+00 3.0000e+00 6.6100e+02
  1.9193e+04 1.5000e+01]
 [2.5100e+02 1.6290e+03 2.0000e+00 0.0000e+00 7.7000e+01 2.7000e+01
  3.1000e+01 5.0000e+00 1.1000e+01 2.6000e+01 2.2000e+01 3.8000e+01
  3.0000e+00 2.1000e+01 1.2310e+03 3.2700e+02 6.0000e+00 1.3800e+02
  7.7730e+03 2.1000e+01]
 [4.1000e+01 7.0000e+00 1.2100e+02 5.0000e+00 6.7700e+02 3.7700e+02
  2.7117e+04 0.0000e+00 0.0000e+00 1.3760e+03 3.0000e+00 4.2440e+03
  3.3000e+01 2.1870e+03 2.5540e+03 4.0000e+00 2.9000e+01 5.0000e+00
  4.0100e+02 1.8137e+04]
 [1.2300e+03 1.0872e+04 1.3000e+01 1.9750e+03 1.4000e+01 0.0000e+00
  2.2000e+02 5.7000e+01 1.1000e+01 7.0000e+00 2.6710e+03 2.8000e+01
  6.9000e+02 8.3000e+01 2.8050e+03 2.1663e+04 5.5800e+02 1.0454e+04
  1.5000e+01 2.1190e+03]
 [3.4800e+02 3.0000e+00 4.8370e+03 6.0000e+00 2.9450e+03 9.7900e+02
  5.0000e+00 3.


[[2.5000e+01 6.0000e+00 1.5000e+01 2.3000e+01 7.0000e+00 3.5600e+02
  0.0000e+00 2.0000e+00 1.4000e+01 5.0000e+00 1.6600e+02 1.0000e+00
  3.0000e+00 4.0000e+00 4.3610e+03 7.0000e+00 5.0000e+00 1.1632e+04
  5.9200e+02 1.1800e+02]
 [2.5500e+02 2.4800e+02 2.8230e+03 2.1300e+02 2.0000e+00 1.9951e+04
  3.0000e+00 2.4000e+02 0.0000e+00 5.1680e+03 4.4600e+02 1.0000e+00
  4.6900e+02 3.0400e+02 5.7000e+01 2.0000e+00 8.2680e+03 1.9000e+01
  3.0000e+00 0.0000e+00]
 [2.1000e+03 8.7360e+03 2.6841e+04 0.0000e+00 6.0200e+02 3.0000e+00
  3.0000e+02 2.5460e+03 3.0000e+00 0.0000e+00 6.0000e+00 1.0000e+01
  2.1600e+04 2.0000e+00 0.0000e+00 1.1620e+03 1.7000e+01 2.0000e+00
  3.3000e+01 4.3600e+03]
 [7.0000e+00 6.0000e+00 4.0000e+00 5.3900e+02 1.2607e+04 1.0710e+04
  2.1000e+01 1.2900e+02 6.0000e+00 4.0000e+00 1.1291e+04 1.0000e+01
  9.0000e+01 3.0700e+02 5.7070e+03 5.0000e+00 2.6460e+03 1.0851e+04
  6.1600e+03 6.6400e+02]
 [2.0000e+00 1.2006e+04 1.6200e+03 1.2310e+03 4.0000e+00 9.7800e+02
  0.0000e+00 6.

...


[[5.3400e+03 5.1880e+03 3.2374e+04 1.6000e+01 4.0000e+00 6.0000e+00
  7.8000e+01 2.1070e+03 6.1150e+03 0.0000e+00 1.6240e+03 1.9912e+04
  6.0650e+03 3.5700e+02 0.0000e+00 2.4000e+01 1.3000e+01 2.2000e+01
  2.4200e+02 2.0000e+00]
 [2.9400e+02 1.0000e+01 9.0000e+00 3.2000e+01 1.5600e+02 3.2040e+03
  5.0000e+00 6.0000e+00 4.0000e+00 1.4160e+03 1.1000e+01 4.0000e+00
  0.0000e+00 2.5000e+01 1.3300e+02 1.4570e+03 0.0000e+00 2.3000e+01
  2.7500e+02 5.8030e+03]
 [7.0000e+00 1.0000e+01 1.0183e+04 2.0000e+00 1.4700e+02 4.4500e+02
  2.0000e+00 1.9590e+03 1.3000e+01 1.5000e+01 2.3658e+04 1.3000e+01
  6.3810e+03 2.0000e+00 2.6000e+01 8.0000e+00 3.0000e+00 4.2830e+03
  1.3298e+04 5.0000e+00]
 [5.5200e+02 1.0000e+01 1.4500e+02 4.0000e+01 4.8000e+01 2.4330e+03
  3.5100e+02 1.0000e+01 1.9100e+03 3.0000e+01 5.0000e+00 1.0180e+03
  4.3000e+01 4.9000e+01 1.8590e+03 2.0540e+03 8.6000e+01 5.3000e+01
  1.4000e+01 1.6800e+02]
 [1.8000e+01 1.0000e+00 4.5000e+02 8.2550e+03 3.8700e+02 1.9700e+03
  7.0000e+00 1.


[[5.4600e+02 1.4990e+03 8.0000e+00 7.4200e+02 6.0000e+00 4.0000e+00
  9.0000e+00 9.1960e+03 1.6000e+01 1.0270e+03 1.8120e+03 9.7700e+02
  3.1910e+03 3.1000e+01 1.0200e+03 3.1200e+02 4.3900e+02 1.0004e+04
  6.0000e+00 3.2700e+02]
 [4.3000e+01 4.1200e+02 1.3150e+03 3.0000e+00 0.0000e+00 5.8000e+01
  8.8630e+03 1.0990e+03 3.9600e+02 0.0000e+00 3.0000e+00 2.3000e+01
  5.7000e+01 5.2830e+03 3.0000e+00 1.3774e+04 3.0000e+00 2.4000e+01
  3.3230e+04 1.2010e+03]
 [1.1000e+01 6.7000e+01 2.7000e+01 2.0000e+00 4.8000e+01 5.0000e+01
  5.0000e+00 3.4190e+03 1.3750e+03 4.2500e+02 2.9000e+01 1.8300e+02
  1.0600e+03 9.6300e+02 0.0000e+00 4.0000e+00 6.0000e+00 2.0670e+03
  3.0000e+00 4.3000e+01]
 [7.0000e+01 7.0000e+00 6.8000e+01 9.1470e+03 5.1000e+01 5.1000e+01
  2.4786e+04 8.6400e+02 4.0000e+00 1.2370e+03 1.9000e+03 5.3000e+01
  3.6000e+01 5.0000e+00 1.6490e+03 1.0000e+00 6.7500e+02 1.3400e+02
  6.0000e+00 1.6910e+03]
 [5.7020e+03 2.2000e+01 3.8080e+03 2.9000e+01 3.0600e+02 1.2521e+04
  3.0000e+00 4.


[[7.0000e+00 4.5000e+01 2.1970e+03 6.0000e+00 2.3300e+02 3.4500e+02
  7.5200e+02 2.0000e+00 1.4700e+02 9.0900e+02 5.6000e+01 1.3000e+01
  5.1500e+02 1.2000e+01 2.0000e+00 2.9741e+04 1.9700e+02 1.8020e+03
  1.2840e+03 6.0000e+00]
 [2.8100e+02 6.3000e+02 2.0000e+00 2.0000e+00 0.0000e+00 1.3820e+03
  3.2230e+03 8.8000e+01 3.5000e+01 7.8000e+01 1.4000e+02 9.0320e+03
  4.6700e+02 4.2000e+01 2.6301e+04 0.0000e+00 6.0000e+00 4.0000e+00
  3.5600e+02 1.0090e+03]
 [3.0000e+00 7.4000e+01 8.2460e+03 6.3500e+02 2.5000e+01 3.5800e+02
  1.0760e+03 3.0000e+00 3.4900e+02 1.6460e+03 7.5000e+01 1.1232e+04
  4.0000e+00 2.0000e+00 5.0000e+00 7.1860e+03 2.0000e+00 1.3000e+01
  6.0900e+02 0.0000e+00]
 [1.6000e+01 1.7560e+03 3.4200e+02 1.1712e+04 9.0000e+00 4.9560e+03
  4.0000e+00 3.7600e+02 8.7020e+03 2.0000e+00 1.9350e+03 2.4000e+01
  1.0250e+03 4.0000e+01 0.0000e+00 1.2350e+03 6.2790e+03 1.5500e+03
  3.8300e+02 2.3000e+01]
 [1.1000e+01 4.0000e+00 7.0000e+00 5.0000e+00 2.2200e+02 2.3000e+01
  1.0000e+00 5.


[[9.0000e+00 8.0000e+00 2.7200e+02 2.3000e+01 2.0000e+00 7.0000e+00
  1.4000e+01 1.0133e+04 5.0000e+00 3.1040e+03 2.8000e+01 5.1000e+01
  2.0000e+00 2.0330e+03 1.5670e+03 2.0000e+01 2.2264e+04 2.3000e+01
  7.6140e+03 4.3900e+02]
 [6.2000e+01 6.5600e+02 1.4100e+02 1.2033e+04 4.0000e+01 2.0000e+00
  0.0000e+00 7.0000e+00 9.0000e+00 7.9000e+01 1.6490e+03 2.9200e+02
  0.0000e+00 4.0000e+00 2.9870e+03 9.0000e+00 5.0000e+00 0.0000e+00
  4.0000e+00 1.0440e+04]
 [5.0000e+00 5.9000e+01 5.9400e+02 4.7500e+02 1.2200e+02 1.9144e+04
  9.9800e+02 5.4000e+01 2.2200e+02 3.0000e+00 8.0000e+00 8.0000e+00
  4.0000e+00 3.4000e+01 3.0000e+00 9.3000e+02 9.0000e+00 0.0000e+00
  1.3000e+01 3.2810e+03]
 [5.9700e+02 0.0000e+00 2.2300e+02 2.2000e+01 8.8000e+02 3.5800e+02
  7.0000e+00 2.7370e+03 6.1100e+03 3.3000e+01 2.0000e+00 2.0000e+00
  1.1000e+01 8.2000e+01 0.0000e+00 1.3240e+03 1.0578e+04 4.1000e+01
  2.2020e+03 2.0000e+00]
 [4.3810e+03 5.8780e+03 3.3210e+03 3.0000e+00 4.3460e+03 3.9200e+02
  7.5200e+02 1.

KeyboardInterrupt: 

'<eos>'